In [1]:
import os 
import ee
import geemap
import numpy as np
import tqdm
import cv2

In [2]:
Map = geemap.Map()
# Import the NLCD collection.
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')

# Filter the collection to the 2016 product.
nlcd2019 = dataset.filter(ee.Filter.eq('system:index', '2019')).first()

# Select the land cover band.
landcover_2019 = nlcd2019.select('landcover')

# Display land cover on the map.
Map.setCenter(-95, 38, 5)
Map.addLayer(landcover_2019, None, 'Landcover')
Map

Map(center=[38, -95], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [3]:
lon = -95.43495
lat = 29.75765
point = ee.Geometry.Point(lon, lat)
dataset = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
image = dataset.filter(ee.Filter.eq('system:index', '2019')).first()
info = image.reduceRegion(ee.Reducer.first(), point, scale=30).getInfo()
pixel_value = info['landcover']
print(pixel_value)

90


It works! Note that lon, lat value of 5 decimal points (0.00001 deg, or 1m) lead to accurate result